In [4]:
import numpy as np 

from scipy.linalg import expm, sinm, cosm

import time

In [5]:

def func1( x, sigma_sq, n): 
    
    noise = np.random.normal(0, np.sqrt( sigma_sq ) )
    
#     A = np.linspace(1,n**2, n)
    
    x = np.sum( np.cos( x ) ) + np.exp( x[0] * x[1]  ) 
    
    return x + noise 
    
    
def func2( x, sigma_sq, n): 
    
    noise = np.random.normal(0, np.sqrt( sigma_sq ) )
        
    x = np.sum( np.sin(x ) ) + np.sum( np.cos(x - np.array( [1]* n ) ) ) + np.exp( x[0] * x[1] + x[1] ) 
    
    return x + noise 


def func( x, y, sigma_sq = 0): 
    
    noise = np.random.normal(0, np.sqrt( sigma_sq ) ) 
    
    return np.exp(x*y) 

In [6]:
def exp_map(x,u): 
    
#     return ( x[0] * np.exp(u[0]) , x[1] * np.exp( u[1] ) )
    return ( x[0] * np.exp(u[0]**3 - u[1]) , x[1] * np.exp( u[1] ) ) 

In [7]:
# def exp_map(x,u): # exponential map for Poincare half plane
    
#     # we ignore the case u[0] = 0
    
#     1./x[1]**2
        
        
    
# #     return ( x[0] * np.exp(u[0]) , x[1] * np.exp( u[1] ) )
#     return ( x[0] * np.exp(u[0]**2 - u[1]) , x[1] * np.exp( u[1] ) )

In [8]:
# def exp_map(x,u):
    
#     return ( x[0] + u[0] , x[1] + u[1] ) 

In [9]:
DELTAs = [0.1,0.01,0.001,0.0001,0.00001] # deltas

Hs = []

x = (1, 1)

for d in DELTAs:
        
    p = exp_map(x,(d+d,0) )
    f1 = func( p[0], p[1] )
    
    p = exp_map(x,(d-d,0) )
    f2 = func( p[0], p[1] )
    
    p = exp_map(x,(d-d,0) )
    f3 = func( p[0], p[1] )
    
    p = exp_map(x,(-d-d,0) )
    f4 = func( p[0], p[1] )
    
    H11 = (f1 - f2 - f2 + f4)/4/d**2
    
    
    
    p = exp_map(x,(d,d) )
    f1 = func( p[0], p[1] )
    
    p = exp_map(x,(d,-d) )
    f2 = func( p[0], p[1] )
    
    p = exp_map(x,(-d,d) )
    f3 = func( p[0], p[1] )
    
    p = exp_map(x,(-d,-d) )
    f4 = func( p[0], p[1] )
    
    H12 = (f1 - f2 - f2 + f4)/4/d**2
    
    
    
    
    p = exp_map(x,(d,d) )
    f1 = func( p[0], p[1] )
    
    p = exp_map(x,(d,-d) )
    f2 = func( p[0], p[1] )
    
    p = exp_map(x,(-d,d) )
    f3 = func( p[0], p[1] )
    
    p = exp_map(x,(-d,-d) )
    f4 = func( p[0], p[1] )
    
    H21 = (f1 - f2 - f2 + f4)/4/d**2
    
    
    
    
    p = exp_map(x,(0,d+d) )
    f1 = func( p[0], p[1] )
    
    p = exp_map(x,(0,d-d) )
    f2 = func( p[0], p[1] )
    
    p = exp_map(x,(0,d-d) )
    f3 = func( p[0], p[1] )
    
    p = exp_map(x,(0,-d-d) )
    f4 = func( p[0], p[1] )
    
    H22 = (f1 - f2 - f2 + f4)/4/d**2
    
    
    
    Hs.append( [[H11, H12], [H21, H22]] )

In [10]:
Hs

[[[0.008698849801203055, -0.13591420468476165], [-0.13591420468476165, 0.0]],
 [[8.698519682326378e-07, -0.01359140914147794], [-0.01359140914147794, 0.0]],
 [[1.1102230246251565e-10, -0.0013591407999058447],
  [-0.0013591407999058447, 0.0]],
 [[1.1102230246251565e-08, -0.00013592460490485792],
  [-0.00013592460490485792, 0.0]],
 [[0.0, -1.5543122344752188e-05], [-1.5543122344752188e-05, 0.0]]]

# timing experiments

In [11]:
def new_est_timing( func, n, noise_var = 0.0025, delta = 0.05 , m = 1, rep = 1 ):

    
    sampling_times = []
    
    eval_times = []
    
    comp_times = []
    
    
    for _ in range(rep):  
        
        start_sampling = time.time()

        for _ in range(m): 

            v = np.random.normal(0, 1, n )
#             v = np.diag(v)
            v = v / np.linalg.norm(v) 

            w = np.random.normal(0, 1, n )
#             v = np.diag(v)
            w = w / np.linalg.norm(w) 
            
            del v; del w

        end_sampling = time.time() 

        time_sampling = end_sampling - start_sampling 

        sampling_times.append(time_sampling )

        start_eval = time.time() 

        for _ in range(m): 

            v = np.random.normal(0, 1, n )
#             v = np.diag(v)
            v = v / np.linalg.norm(v) 

            w = np.random.normal(0, 1, n )
            w = w / np.linalg.norm(w) 

            f1 = func( delta*v + delta*w, noise_var, n ) 
            f2 = func( delta*v - delta*w, noise_var, n ) 
            f3 = func( -delta*v + delta*w, noise_var, n ) 
            f4 = func( -delta*v - delta*w, noise_var, n ) 
            
            del v; del w; del f1; del f2; del f3; del f4
#             gc.collect()

        end_eval = time.time() 

        time_eval = end_eval - start_eval 

        eval_times.append(time_eval)

        start_comp = time.time()

        for _ in range(m):

            v = np.random.normal(0, 1, n )
#             v = np.diag(v)
            v = v / np.linalg.norm(v) 

            w = np.random.normal(0, 1, n )
#             w = np.diag(w)
            w = w / np.linalg.norm(w) 

            f1 = func( delta*v + delta*w, noise_var, n ) 
            f2 = func( delta*v - delta*w, noise_var, n ) 
            f3 = func( -delta*v + delta*w, noise_var, n ) 
            f4 = func( -delta*v - delta*w, noise_var, n ) 

            grad_est = (f1 - f2 - f3 + f4) / ( 8 * delta**2 ) * n**2 * ( np.outer(v,w) + np.outer(w,v) ) 
            
            del v; del w; del f1; del f2; del f3; del f4; del grad_est
            
#             gc.collect()

        end_comp = time.time() 

        time_comp = end_comp - start_comp

        comp_times.append(time_comp)

#         avg_est = avg_est + grad_est  

#     avg_est = avg_est / m 
    
    return sampling_times, eval_times, comp_times 

In [18]:
def steins_est_timing( func, n, noise_var = 0.0025, delta = 0.01 , m = 100, rep = 100 ):

#     avg_est = 0 

    sampling_times = []
    
    eval_times = []
    
    comp_times = [] 
    
    for _ in range(rep):
    
        start_sampling = time.time()

        for _ in range(m): 

            v = np.random.normal(0, 1, n )
#             v = np.diag(v)
            
            del v 

        end_sampling = time.time() 
        time_sampling = end_sampling - start_sampling 
        sampling_times.append(time_sampling ) 
        
        # -------------------------- 
        
        start_eval = time.time() 
    
        for _ in range(m): 

            v = np.random.normal(0, 1, n ) 
#             v = np.diag(v) 

            f1 = func( delta*v, noise_var, n ) 
            f2 = func( -delta*v, noise_var, n ) 
            f3 = func( np.zeros(n), noise_var, n ) 
            
        end_eval = time.time() 
        time_eval = end_eval - start_eval 
        eval_times.append( time_eval ) 
        
        #----------------------------------- 
        
        start_comp = time.time()
        
        for _ in range(m):

            v = np.random.normal(0, 1, n )
#             v = np.diag(v) 

            f1 = func( delta*v, noise_var, n ) 
            f2 = func( -delta*v, noise_var, n ) 
            f3 = func( np.zeros(n), noise_var, n ) 

            D2 = (f1 - 2*f3 + f2) / (2*delta**2) 

            grad_est = ( np.outer(v,v) - np.identity(n) ) * D2 
            
        end_comp = time.time()
        
        time_comp = end_comp - start_comp 
        comp_times.append( time_comp ) 

#         avg_est = avg_est + grad_est  
        
#     avg_est = avg_est / m 
    
    return sampling_times, eval_times, comp_times 

In [60]:
sampling_times, eval_times, comp_times = new_est_timing( func1, 8 , m = 10000, rep = 10 ) 

sampling_times_stein, eval_times_stein, comp_times_stein = steins_est_timing( func1, 8 , m = 10000, rep = 10 ) 

In [61]:
print( np.mean( sampling_times ), np.std(sampling_times) )

0.15797748565673828 0.0031285555349549717


In [62]:
print( np.mean( eval_times ), np.std( eval_times) )

0.5762594223022461 0.003590354661671788


In [63]:
print( np.mean( comp_times ), np.std( comp_times) )

0.6977411985397339 0.0040099644223627155


In [64]:
print( np.mean( sampling_times_stein ), np.std( sampling_times_stein ) )

0.025734615325927735 0.0011615425578557481


In [65]:
print( np.mean( eval_times_stein ), np.std( eval_times_stein ) )

0.3019917726516724 0.0023926765862216873


In [66]:
print( np.mean( comp_times_stein ), np.std( comp_times_stein ) )

0.42216830253601073 0.0027861916419257542


In [67]:
sampling_times_stein

[0.024933576583862305,
 0.025930404663085938,
 0.024931669235229492,
 0.028925418853759766,
 0.024938106536865234,
 0.024934053421020508,
 0.025928735733032227,
 0.02593064308166504,
 0.024962663650512695,
 0.02593088150024414]

In [68]:
comp_times_stein

[0.4188835620880127,
 0.42087650299072266,
 0.42386674880981445,
 0.422870397567749,
 0.4218721389770508,
 0.4228677749633789,
 0.427854061126709,
 0.4228396415710449,
 0.4168820381164551,
 0.4228701591491699]

In [39]:
import sys

sys.version

'3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)]'

In [45]:
time.time()

1659032643.4402635

In [46]:
time.time()

1659032650.6711578